# Making Necessary Imports:

In [521]:
!pip install dython

In [522]:
import pandas as pd;

import numpy as np;

from sklearn.preprocessing import LabelEncoder;
from sklearn.decomposition import PCA;

import matplotlib.pyplot as plt;

import seaborn as sns;

from sklearn import preprocessing;
from sklearn.preprocessing import OneHotEncoder;
from sklearn.neighbors import KNeighborsClassifier;
from sklearn.model_selection import train_test_split;

from dython import nominal;

# configs:
pd.options.display.max_columns = None;

# Initializing the Constants to be Used in The Program: 

In [523]:
# "adult.data" file.
inputDataURL = "https://raw.githubusercontent.com/Aditya-163/MLDA-1-Project-1/main/Data/adult.data"; 
# "adult.test" file.
testDataURL = "https://raw.githubusercontent.com/Aditya-163/MLDA-1-Project-1/main/Data/adult.test";

# Attributes in the data.
allAttrs = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","income"]; 
xAttrs = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country"];
yAttrs = ["income"];

# Numerical Attributes:
numXAttrs = ["age","fnlwgt","education-num","capital-gain","capital-loss","hours-per-week"];

# Nominal Attributes:
nominalXAttrs = ["workclass","education","marital-status","occupation","relationship","race","sex","native-country"];

# Reading The Required Data:

In [ ]:
# Training Data.
inputData = pd.read_csv(inputDataURL, names=colNames, na_values=" ?");
inputData = inputData.dropna();
print("Some rows from the input data are shown below:");
print(inputData.head());

# Test Data.
testData = pd.read_csv(testDataURL, names=colNames, na_values=" ?");
testData = testData.dropna();

# Concataneted Data
data = pd.concat([inputData,testData]);

# Some Visualizations From Data:

# Pre-Processing The Data:

### Normalizing Numerical Data:

In [484]:
data[numXAttrs] = preprocessing.MinMaxScaler().fit_transform(concatData[numXAttrs]);

### Splitting The Input and Output Parts in the data:

In [485]:
X = data[xAttrs];
X.reset_index(drop=True, inplace=True);

Y = data[yAttrs];
Y = Y.replace(to_replace =[" <=50K"," <=50K."], value ="<=50K");
Y = Y.replace(to_replace =[" >50K"," >50K."], value =">50K");
Y.reset_index(drop=True, inplace=True);

### PCA On Numerical Data:

In [ ]:
Ypca = Y;

pca = PCA(0.90);
pca.fit(X[numXAttrs]);
XDimReduced = pca.transform(X[numAttrs]);

print("Number of PCs for explaining the required variance is:");
print(XDimReduced.shape[1]);

# Scree Plot!
print("The Scree Plot:");
plt.plot(pca.explained_variance_ratio_);
plt.ylabel('Explained Variance');
plt.xlabel('Components');
plt.show();


Xpca = pd.DataFrame({'PC1':XDimReduced[:,0], 'PC2':XDimReduced[:,1], 'PC3':XDimReduced[:,2], 'PC4':XDimReduced[:,3], 'PC5':XDimReduced[:,4]});

Xpca = pd.concat([X,Xpca],axis=1);
XpcaAttrs = nominalXAttrs + ["PC1","PC2","PC3","PC4","PC5"];
Xpca = Xpca[XpcaAttrs];
print("Input data after reducing dimensions using PCA:");
print(Xpca.head());

### Correlation Between Nominal Variables:

In [ ]:
XWithNominalAttrs = X[nominalXAttrs];
correlationMatrixForNominalData = nominal.associations(XWithNominalAttrs, nominal_columns='all'); # Uses Correlation Ratio for finding the correlation!
print("The above heatmap in matrix form:");
print(correlationMatrixForNominalData);

# sex (gender) and relationship have a correlation of 0.65, hence eliminating relationship attribute from our data!
XpcaAttrs.remove('relationship');
Xpca = Xpca[XpcaAttrs];
XpcaAttrs.append('relationship');
print("Input data after reducing dimensions using PCA and Correlation:");
print(Xpca.head());

# Ref: https://github.com/shakedzy/dython/issues/2

### Encoding Ordinal Data:

In [ ]:
X = pd.get_dummies(X,drop_first=True);
Y = pd.get_dummies(Y,drop_first=True);
X.reset_index(drop=True, inplace=True);
Y.reset_index(drop=True, inplace=True);
print("X shape:");
print(X.shape);
print("X dtypes:");
print(X.dtypes);
print("Y shape:");
print(Y.shape);
print("Y dtypes:");
print(Y.dtypes);

Xpca = pd.get_dummies(Xpca,drop_first=True);
Ypca = pd.get_dummies(Ypca,drop_first=True);
Xpca.reset_index(drop=True, inplace=True);
Ypca.reset_index(drop=True, inplace=True);
print("Xpca shape:");
print(Xpca.shape);
print("Xpca dtypes:");
print(Xpca.dtypes);
print("Ypca shape:");
print(Ypca.shape);
print("Ypca dtypes:");
print(Ypca.dtypes);

# KNN (Without Using PCA and Correlation):

In [ ]:
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size = 0.33, random_state=42);

kValues = [50,100,250,500];
accuracies = list();

for k in kValues:
  KNN = KNeighborsClassifier(n_neighbors=k);
  KNN.fit(XTrain, YTrain);
  accuracies.append(knn.score(XTest, YTest));

fig, ax = plt.subplots(1)

xAxisData = np.array(kValues)
yAxisData = np.array(accuracies);
yAxisData = yAxisData*100;

ax.plot(xAxisData,yAxisData);
 
ax.set_xticks(kValues);
ax.set_xticklabels(kValues);

# KNN (With Using PCA and Correlation):



In [ ]:
XpcaTrain, XpcaTest, YpcaTrain, YpcaTest = train_test_split(Xpca, Ypca, test_size = 0.33, random_state=42);

print(XpcaTrain.shape);
print(XpcaTest.shape);
print(YpcaTrain.shape);
print(YpcaTest.shape);


kValues = [50,100,250,500];
accuracies = list();

for k in kValues:
  KNN = KNeighborsClassifier(n_neighbors=k);
  KNN.fit(XpcaTrain, YpcaTrain);
  accuracies.append(knn.score(XpcaTest, YpcaTest));

fig, ax = plt.subplots(1)

xAxisData = np.array(kValues)
yAxisData = np.array(accuracies);
yAxisData = yAxisData*100;

ax.plot(xAxisData,yAxisData);
 
ax.set_xticks(kValues);
ax.set_xticklabels(kValues);